In [14]:
import pandas as pd
import seaborn as sns
import numpy as np
import os

In [15]:
### 함수 정의 ###

# train/test 데이터프레임과 stores 데이터프레임을 입력받아 데이터 머지 후, 반환하는 함수
def train_store_mearge(input, stores):
    return pd.merge(input, stores, how='left', on='store_nbr')

# oil 데이터의 빠진 날짜와 dcoilwtico 데이터를 interpolation으로 채운 dataframe을 반환하는 함수
def fill_oil_data(df):
    # date 열을 인덱스로 설정
    df.set_index('date', inplace=True)
    # 전체 날짜 범위를 생성하여 누락된 날짜를 추가 (min에서 max 사이의 모든 날짜)
    df = df.reindex(pd.date_range(start=df.index.min(), end=df.index.max()))
    # price 값을 선형 보간법으로 채우기
    df['dcoilwtico'] = df['dcoilwtico'].interpolate()
    # 인덱스를 다시 date 열로 복원
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'date'}, inplace=True)
    return df

# 날짜와 holiday list를 입력받아, 해당 날짜가 holiday 인지 판별하여 True/False 값 반환하는 함수
def is_holiday(dt, hdts):
    result = dt in hdts.values
    return result

# 날짜를 입력받아 해당 날짜가 workday 인지 weekend인지 판별하고 결과를 'workday' or 'weekend'로 반환하는 함수
def normal_day_type(dt):
    if dt.weekday() >= 5:
        return 'Weekend'
    else:
        return 'Work Day'
    
# ['date'], ['city'], ['state'] 그리고 df_holiday_events 데이터프레임을 입력받아 date.holiday type을 반환하는 함수
def date_type(dt, city, state, dhe):
    dt_type = ""
    # 입력받은 날짜가 df_holiday_events에 포함되어 있는 경우, lcoal->regional->national holidays 순서로 해당되는 type을 확인
    if is_holiday(dt, dhe['date']):
        holidays = dhe[dhe['date'] == dt]
        for _, holiday in holidays.iterrows():
            # Local인 경우, city 비교
            if holiday['locale'] == 'Local' and city == holiday['locale_name']:
                dt_type = holiday['type']
                break
            elif holiday['locale'] == 'Regional' and state == holiday['locale_name']:
                dt_type = holiday['type']
            elif holiday['locale'] == 'National' and dt_type == "":
                dt_type = holiday['type']
    if dt_type == "":
        dt_type = normal_day_type(dt)
    return dt_type

def pre_proecess(df, df_stores, df_holiday_events, df_oil, purpose='train'):
    # df_oil_filled 데이터의 빠진 날짜 및 dcoilwtico 데이터 채우기
    df_oil_filled = df_oil.copy()
    df_oil_filled = fill_oil_data(df_oil_filled)
    # 데이터의 첫 날 (2013-01-01)의 dcoilwtico 데이터는 누락되어 있으므로, 그 다음날의 dcoilwtico 값과 동일한 것으로 가정
    df_oil_filled.iloc[0, 1] = df_oil_filled.iloc[1, 1]  
    # train 데이터에 dcoilwtico 값 추가
    df_train_stores = train_store_mearge(df, df_stores)

    # 헤더만 쓰기 위해 첫 번째 빈 데이터프레임을 저장
    with open ('temp_data.csv', 'w') as f:
        pd.DataFrame(columns=df_train_stores.columns.tolist()+['date_type']).to_csv(f, index=False)

    # 데이터 프레임을 chunk_size씩 데이터를 확인하면서, holiday_events 데이터를 기준으로 date_type을 확인해서 train 데이터에 추가
    chunk_size = 1000
    end_row = len(df_train_stores)

    for i in range(0, end_row + chunk_size, chunk_size):
        if i > end_row:
            print(f"{idx} rows are porcessed.")
            break
        else:
            chunk = df_train_stores.iloc[i:i + chunk_size].copy()  # i에서 i + chunk_size까지 슬라이싱
            # date_type을 구분
            for idx, row in chunk.iterrows():
                chunk.at[idx,'date_type']=date_type(row['date'], row['city'], row['state'], df_holiday_events)
        print(f"{i+1} row(s) are porcessed.")
        # 결과를 파일에 추가 모드로 저장
        chunk.to_csv('temp_data.csv', mode='a', header=False, index=False)

    # train data에 stroes 및 date_type 정보가 추가된 데이터를 temp_data 데이터프레임에 임시 저장
    temp_data = pd.read_csv('temp_data.csv', parse_dates =['date'])
    # (삭제예정) - temp_data['date'] = pd.to_datetime(temp_data['date'])
    # temp_data에 df_oil_filled 데이터를 기준으로 dcoilwtico 데이터를 추가해서, train_data 데이터 프레임에 저장
    train_data = pd.merge(temp_data, df_oil_filled, how='left', on='date')
    
    # 처리한 데이터를 저장할 csv 파일명을 purpose 기준으로 생성하고, train_data를 해당 csv 파일에 저장
    filename = purpose+".csv"
    train_data.to_csv(filename, index=False)

    print("Data preprocess is completed, and the result is saved to train_data.csv.")
    print(train_data.info())

    # temp_data.csv 파일 삭제
    if os.path.exists('temp_data.csv'):
        os.remove('temp_data.csv')
        print(f"temp_data.csv 파일이 삭제되었습니다.")
    else:
        print(f"temp_data.csv 파일이 존재하지 않습니다.")

    return True


In [16]:
# train, store, holiday_events, oil, test 데이터 로드 / df_test 데이터에 대한 transaction 데이터는 존재하지 않기 때문에 transcation 데이터는 모델 학습에 사용하지 않음
path = 'store-sales-time-series-forecasting/'
df_train = pd.read_csv(path + 'train.csv',parse_dates =['date'])
df_stores = pd.read_csv(path + 'stores.csv')
df_holiday_events = pd.read_csv(path + 'holidays_events.csv',parse_dates =['date'])
df_oil = pd.read_csv(path + 'oil.csv',parse_dates =['date'])
df_test = pd.read_csv(path + 'test.csv',parse_dates =['date'])

In [17]:
pre_proecess(df = df_train, df_stores=df_stores, df_holiday_events=df_holiday_events, df_oil=df_oil, purpose='train')
pre_proecess(df = df_test, df_stores=df_stores, df_holiday_events=df_holiday_events, df_oil=df_oil, purpose='test')


1 row(s) are porcessed.
1001 row(s) are porcessed.
2001 row(s) are porcessed.
3001 row(s) are porcessed.
4001 row(s) are porcessed.
5001 row(s) are porcessed.
6001 row(s) are porcessed.
7001 row(s) are porcessed.
8001 row(s) are porcessed.
9001 row(s) are porcessed.
10001 row(s) are porcessed.
11001 row(s) are porcessed.
12001 row(s) are porcessed.
13001 row(s) are porcessed.
14001 row(s) are porcessed.
15001 row(s) are porcessed.
16001 row(s) are porcessed.
17001 row(s) are porcessed.
18001 row(s) are porcessed.
19001 row(s) are porcessed.
20001 row(s) are porcessed.
21001 row(s) are porcessed.
22001 row(s) are porcessed.
23001 row(s) are porcessed.
24001 row(s) are porcessed.
25001 row(s) are porcessed.
26001 row(s) are porcessed.
27001 row(s) are porcessed.
28001 row(s) are porcessed.
29001 row(s) are porcessed.
30001 row(s) are porcessed.
31001 row(s) are porcessed.
32001 row(s) are porcessed.
33001 row(s) are porcessed.
34001 row(s) are porcessed.
35001 row(s) are porcessed.
36001

True

In [18]:
#전처리 후, csv에 저장한 파일을 데이터프레임으로 읽어서 정보 확인

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
print(train_data.info())
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 12 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
 6   city         object 
 7   state        object 
 8   type         object 
 9   cluster      int64  
 10  date_type    object 
 11  dcoilwtico   float64
dtypes: float64(2), int64(4), object(6)
memory usage: 274.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           28512 non-null  int64  
 1   date         28512 non-null  object 
 2   store_nbr    28512 non-null  int64  
 3   family       28512 non-null  object 
 4   onpromotion  28512 non-null  int64  
 5   city         28512 non-null  object 
 6   stat

In [19]:
train_data.isnull().value_counts()

id     date   store_nbr  family  sales  onpromotion  city   state  type   cluster  date_type  dcoilwtico
False  False  False      False   False  False        False  False  False  False    False      False         3000888
Name: count, dtype: int64

In [20]:
test_data.isnull().value_counts()

id     date   store_nbr  family  onpromotion  city   state  type   cluster  date_type  dcoilwtico
False  False  False      False   False        False  False  False  False    False      False         28512
Name: count, dtype: int64

In [21]:
train_data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 12 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   id           3000888 non-null  int64  
 1   date         3000888 non-null  object 
 2   store_nbr    3000888 non-null  int64  
 3   family       3000888 non-null  object 
 4   sales        3000888 non-null  float64
 5   onpromotion  3000888 non-null  int64  
 6   city         3000888 non-null  object 
 7   state        3000888 non-null  object 
 8   type         3000888 non-null  object 
 9   cluster      3000888 non-null  int64  
 10  date_type    3000888 non-null  object 
 11  dcoilwtico   3000888 non-null  float64
dtypes: float64(2), int64(4), object(6)
memory usage: 274.7+ MB
